In [147]:
import csv
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from datetime import datetime
from scipy import stats
import math

In [148]:
ps = pd.read_csv('patch_seq_log_mouse.csv')

In [149]:
#ps.head()

In [150]:
ps = ps[["Date", "User", "Post patch?", "Post patch pipette R", "Nucleus sucked in?", "Notes","Internal version", "PCR cycles", "SM_QC_PF", "Resolution index", "Bad dates"]]
#ps.dropna(subset = ["SM_QC_PF", "Post patch?"], inplace = True)
ps.dropna(subset = ["Post patch?"], inplace = True)
ps["User"] = ps["User"].astype("category")
ps["Post patch?"] = ps["Post patch?"].astype("category")
ps["SM_QC_PF"] = ps["SM_QC_PF"].astype("category")
ps = ps[ps["Bad dates"] != 'x']
ps = ps[ps["SM_QC_PF"] != "cnt"]
ps = ps[ps["PCR cycles"] == 21]
ps = ps.drop('Bad dates', 1)
ps["Date"] = pd.to_datetime(ps["Date"], format = '%y%m%d')
#ps["PCR cycles"].value_counts()
ps

,Date,User,Post patch?,Post patch pipette R,Nucleus sucked in?,Notes,Internal version,PCR cycles,SM_QC_PF,Resolution index
4968,2017-04-19,P2,Outside-Out,NaN,NaN,NaN,AiV2,21.0,fail,NaN
4969,2017-04-19,P2,Outside-Out,NaN,NaN,NaN,AiV2,21.0,pass,0.923406
4970,2017-04-19,P2,No-Seal,NaN,NaN,NaN,AiV2,21.0,fail,NaN
4971,2017-04-19,P2,Nucleated,NaN,NaN,NaN,AiV2,21.0,pass,0.000000
4972,2017-04-19,P2,Outside-Out,NaN,NaN,NaN,AiV2,21.0,pass,0.322631
4973,2017-04-19,P2,Nucleated,NaN,NaN,NaN,AiV2,21.0,pass,0.322631
4974,2017-04-19,P2,No-Seal,NaN,NaN,NaN,AiV2,21.0,pass,0.000000
4975,2017-04-19,P2,Nucleated,NaN,NaN,NaN,AiV2,21.0,pass,NaN
4978,2017-04-19,P6,No-Seal,NaN,NaN,NaN,AiV2,21.0,fail,NaN
4979,2017-04-19,P6,No-Seal,NaN,NaN,NaN,AiV2,21.0,pass,0.000000


#### defining counting functions

In [151]:
def get_count(x):    
    count = x["Post patch?"].count()
    return count

def calc_pct(passing,total):
    pct = float(get_count(passing)) / get_count(total)
    return pct

In [152]:
#total = ps["Post patch?"].count()
total = get_count(ps)
total
ps['Post patch?'].value_counts()

Nucleated             755
No-Seal               556
Partial-Nucleus       204
nucleus_present        93
Outside-Out            92
nucleus_absent         76
Entire-Cell            17
no_nucleus_visible      6
nucleus_visible         4
entire_cell             3
missing                 1
Outside-Out?            0
Nucleus                 0
outside-out             0
Outside-out             0
no -seal                0
no-seal                 0
nr                      0
nucelated               0
Entire cell             0
Name: Post patch?, dtype: int64

#### defining filtering functions

In [153]:
#input is a dataframe

def nucleated(x):
    nucleus = x[(x["Post patch?"] == "Nucleated") | (((x["Post patch?"] == "nucleus_visible") | (x["Post patch?"] == "nucleus_present")) & (x["Post patch pipette R"] >= 501))]
    return nucleus

def partial_nucleated(y):
    partial = y[(y["Post patch?"] == 'Partial-Nucleus') | (((y['Post patch?'] == 'nucleus_present') | (y['Post patch?'] == 'nucleus_visible')) & (y["Post patch pipette R"] <= 500))]
    return partial

def outside_out(z):
    outside = z[(z["Post patch?"] == 'Outside-Out') | (((z['Post patch?'] == 'nucleus_absent') | (z['Post patch?'] == 'no_nuclues_visible')) & (z['Post patch pipette R'] >= 501))]
    return outside

def no_seal(w): 
    no = w[(w["Post patch?"] == 'No-Seal') | (((w['Post patch?'] == 'nucleus_absent') | (w['Post patch?'] == 'no_nucleus_visible')) & (w['Post patch pipette R'] <= 500))]
    return no
    
def pass_only(df):
    pas = df[df["SM_QC_PF"] == "pass"]
    return pas


# Post patch condition percentages

#### Filtering to nucleated patches only and calculating percent nucleated

In [154]:
#nuc = ps[(ps["Post patch?"] == "Nucleated") | (ps["Post patch?"] == "Partial-Nucleus") | (ps["Post patch?"] == "nucleus_present") | (ps["Post patch?"] == "nucleus_visible")]
#nuc = ps[(ps["Post patch?"] == "Nucleated") | (((ps["Post patch?"] == "nucleus_visible") | (ps["Post patch?"] == "nucleus_present")) & (ps["Post patch pipette R"] >= 500))]
nuc = nucleated(ps)
nuc
#dic = {'Nucleated': 'test',
    #  'nucelated': 'Nucleated'
    # }

#ps['Post patch?'] = ps['Post patch?'].map(lambda x: dic[x])

,Date,User,Post patch?,Post patch pipette R,Nucleus sucked in?,Notes,Internal version,PCR cycles,SM_QC_PF,Resolution index
4971,2017-04-19,P2,Nucleated,NaN,NaN,NaN,AiV2,21.0,pass,0.000000
4973,2017-04-19,P2,Nucleated,NaN,NaN,NaN,AiV2,21.0,pass,0.322631
4975,2017-04-19,P2,Nucleated,NaN,NaN,NaN,AiV2,21.0,pass,NaN
4988,2017-04-20,P1,Nucleated,NaN,NaN,NaN,AiV2,21.0,pass,0.601033
4997,2017-04-20,P1,Nucleated,NaN,NaN,NaN,AiV2,21.0,pass,NaN
5009,2017-04-20,P2,Nucleated,NaN,NaN,NaN,AiV2,21.0,pass,0.678447
5022,2017-04-21,P1,Nucleated,NaN,NaN,"Fluorescence in Pipette,Cell Dimmed",AiV2,21.0,pass,0.795757
5026,2017-04-21,P2,Nucleated,NaN,NaN,Fluorescence in Pipette,AiV2,21.0,pass,1.000000
5027,2017-04-21,P2,Nucleated,NaN,NaN,Fluorescence in Pipette,AiV2,21.0,pass,0.877682
5035,2017-04-21,P2,Nucleated,NaN,NaN,NaN,AiV2,21.0,pass,0.556320


In [155]:
#nuc_count = nuc["Post patch?"].count()
nuc_count = get_count(nuc)
nuc_count

831

In [156]:
#nuc_percent = float(nuc_count)/total
#nuc_percent
nuc_pct = calc_pct(nuc, ps) 
print "nucleated patch %:", nuc_pct

nucleated patch %: 0.459878251245


#### Filtering to partial-nucleated patches only and calculating percent partial-nucleated

In [157]:
part_nuc = partial_nucleated(ps)
part_nuc

,Date,User,Post patch?,Post patch pipette R,Nucleus sucked in?,Notes,Internal version,PCR cycles,SM_QC_PF,Resolution index
4996,2017-04-20,P1,Partial-Nucleus,NaN,NaN,NaN,AiV2,21.0,pass,0.678447
5021,2017-04-21,P1,Partial-Nucleus,NaN,NaN,"Fluorescence in Pipette,Cell Dimmed",AiV2,21.0,pass,1.000000
5047,2017-04-24,P1,Partial-Nucleus,NaN,NaN,Cell Dimmed lots of small bubbles,AiV2,21.0,fail,NaN
5135,2017-04-27,P1,Partial-Nucleus,NaN,NaN,Fluorescence in Pipette lots of small bubbles,AiV2,21.0,fail,NaN
5169,2017-04-28,P2,Partial-Nucleus,NaN,NaN,one tiny bubble formed during expulsion.,AiV2,21.0,pass,1.000000
5171,2017-04-28,P2,Partial-Nucleus,NaN,NaN,no bubbles during expulsion.,AiV2,21.0,pass,1.000000
5188,2017-05-01,P2,Partial-Nucleus,NaN,NaN,Fluorescence in Pipette One big bubble formed ...,AiV2,21.0,pass,NaN
5190,2017-05-01,P2,Partial-Nucleus,NaN,NaN,Fluorescence in Pipette,AiV2,21.0,pass,NaN
5202,2017-05-01,P6,Partial-Nucleus,NaN,NaN,NaN,AiV2,21.0,fail,NaN
5203,2017-05-01,P6,Partial-Nucleus,NaN,NaN,NaN,AiV2,21.0,fail,NaN


In [158]:
part_nuc_pct = calc_pct(part_nuc, ps)
print "partial nucleated patch %:", part_nuc_pct

partial nucleated patch %: 0.124515771998


#### Filtering to outside-out patches only and calculating percentage

#### Filtering to no-seal patches only and calculating percentage

# Calculating nucleated patch percentages by user

In [159]:
P8_total = ps[ps["User"] == 'P8']
P8_total_count = P8_total['Post patch?'].count()
P8_total_count

447

In [160]:
#use post patch nucleated filter
P8_nuc = P8_total[P8_total['Post patch?'] == 'Nucleated']
P8_nuc_count = P8_nuc['Post patch?'].count()
P8_nuc_count

211

In [161]:
float(P8_nuc_count)/P8_total_count

0.4720357941834452

In [162]:
for riguser in ['P1','P2','P6','P8','P9','PA','PB']:
    user_total = ps[ps['User'] == riguser]
    user_total_count = user_total['Post patch?'].count()
    user_nuc = user_total[user_total['Post patch?'] == 'Nucleated']
    user_nuc_count = user_nuc['Post patch?'].count()
    user_percent = float(user_nuc_count)/(user_total_count)
    print riguser,': ', user_percent
    

P1 :  0.342490842491
P2 :  0.482142857143
P6 :  0.369863013699
P8 :  0.472035794183
P9 :  0.450549450549
PA :  0.28125
PB :  0.0888888888889


# Obtaining nucleus dependant on internal solution version

#### Creating dataframe for standard internal only

In [163]:
standard_internal = ps[ps["Internal version"] == "AiV2"]
standard_internal = standard_internal[(standard_internal["User"] == "P2") | (standard_internal["User"] == "P8")]
standard_internal.head()
                       

,Date,User,Post patch?,Post patch pipette R,Nucleus sucked in?,Notes,Internal version,PCR cycles,SM_QC_PF,Resolution index
4968,2017-04-19,P2,Outside-Out,NaN,NaN,NaN,AiV2,21.0,fail,NaN
4969,2017-04-19,P2,Outside-Out,NaN,NaN,NaN,AiV2,21.0,pass,0.923406
4970,2017-04-19,P2,No-Seal,NaN,NaN,NaN,AiV2,21.0,fail,NaN
4971,2017-04-19,P2,Nucleated,NaN,NaN,NaN,AiV2,21.0,pass,0.000000
4972,2017-04-19,P2,Outside-Out,NaN,NaN,NaN,AiV2,21.0,pass,0.322631


In [164]:
#total_standard_internal = standard_internal["Post patch?"].count()
total_standard_internal = get_count(standard_internal)
total_standard_internal

759

#### Creating dataframe for alexa internal only

In [165]:
alexa_internal = ps[ps["Internal version"] == "AiV2-50uM alexa 488"]
alexa_internal = alexa_internal[(alexa_internal["User"] == "P2") | (alexa_internal["User"] == "P8")]
alexa_internal

,Date,User,Post patch?,Post patch pipette R,Nucleus sucked in?,Notes,Internal version,PCR cycles,SM_QC_PF,Resolution index
7094,2017-08-29,P8,Entire-Cell,NaN,NaN,"Fluorescence in Pipette,Cell Dimmed,Cell Shrun...",AiV2-50uM alexa 488,21.0,pass,0.322631
7095,2017-08-29,P8,Partial-Nucleus,NaN,NaN,"Fluorescence in Pipette,Cell Dimmed,Cell Swell...",AiV2-50uM alexa 488,21.0,pass,0.000000
7096,2017-08-30,P8,Partial-Nucleus,NaN,NaN,"Fluorescence in Pipette,Cell Swelled No Bubble...",AiV2-50uM alexa 488,21.0,pass,0.783192
7099,2017-08-30,P8,No-Seal,NaN,NaN,"Fluorescence in Pipette,Cell Shrunk No Bubbles...",AiV2-50uM alexa 488,21.0,pass,0.000000
7101,2017-08-30,P8,No-Seal,NaN,NaN,Fluorescence in Pipette No Bubbles,AiV2-50uM alexa 488,21.0,pass,0.322631
7108,2017-08-31,P2,Nucleated,NaN,NaN,No Bubbles 0.1 units of pressure on expulsion,AiV2-50uM alexa 488,21.0,pass,1.000000
7109,2017-08-31,P2,Nucleated,NaN,NaN,Fluorescence in Pipette No Bubbles 0.1 units o...,AiV2-50uM alexa 488,21.0,pass,1.000000
7110,2017-08-31,P2,Nucleated,NaN,NaN,Fluorescence in Pipette No Bubbles 0.1 units o...,AiV2-50uM alexa 488,21.0,pass,1.000000
7113,2017-08-31,P8,Nucleated,NaN,NaN,"Fluorescence in Pipette,Cell Dimmed,Cell Shrun...",AiV2-50uM alexa 488,21.0,pass,1.000000
7114,2017-08-31,P8,No-Seal,NaN,NaN,Cell Disappeared,AiV2-50uM alexa 488,21.0,pass,0.469928


In [166]:
#total_alexa_internal = alexa_internal["Post patch?"].count()
total_alexa_internal = get_count(alexa_internal)
total_alexa_internal

248

#### Percent of nucleated patches with standard internal

In [167]:
#standard_internal_nuc = standard_internal[(standard_internal["Post patch?"] == "Nucleated") | (standard_internal["Post patch?"] == "Partial-Nucleus") | (standard_internal["Post patch?"] == "nucleus_present")|(standard_internal["Post patch?"] == "nucleus_visible")]
standard_internal_nuc = nucleated(standard_internal)
standard_internal_nuc
#standard_internal_nuc_count = standard_internal_nuc["Post patch?"].count()
standard_internal_nuc_pct = calc_pct(standard_internal_nuc, standard_internal)
standard_internal_nuc_pct
#print "standard internal:" , float(standard_internal_nuc_count)/total_standard_internal



0.5125164690382081

#### Percent of nucleated patched with alexa internal

In [168]:
#alexa_internal_nuc = alexa_internal[(alexa_internal["Post patch?"] == "Nucleated") | (alexa_internal["Post patch?"] == "Partial-Nucleus") | (alexa_internal["Post patch?"] == "nucleus_present")|(alexa_internal["Post patch?"] == "nucleus_visible")]
alexa_internal_nuc = nucleated(alexa_internal)
alexa_internal_nuc
#alexa_internal_nuc_count = alexa_internal_nuc["Post patch?"].count()
alexa_internal_nuc_pct = calc_pct(alexa_internal_nuc, alexa_internal)
alexa_internal_nuc_pct
#print "alexa internal:" , float(alexa_internal_nuc_count)/total_alexa_internal

0.532258064516129

# Rate of SMQC pass by internal solution verion

In [169]:
standard_internal.dropna(subset = ["SM_QC_PF", "Post patch?"], inplace = True)
standard_internal_count = standard_internal["Post patch?"].count()
standard_internal_count

758

In [170]:
standard_internal_pass = standard_internal[standard_internal["SM_QC_PF"] == "pass"]
standard_internal_pass
#standard_internal_pass_count = standard_internal_pass["Post patch?"].count()
#standard_internal_pass_count                                    

,Date,User,Post patch?,Post patch pipette R,Nucleus sucked in?,Notes,Internal version,PCR cycles,SM_QC_PF,Resolution index
4969,2017-04-19,P2,Outside-Out,NaN,NaN,NaN,AiV2,21.0,pass,0.923406
4971,2017-04-19,P2,Nucleated,NaN,NaN,NaN,AiV2,21.0,pass,0.000000
4972,2017-04-19,P2,Outside-Out,NaN,NaN,NaN,AiV2,21.0,pass,0.322631
4973,2017-04-19,P2,Nucleated,NaN,NaN,NaN,AiV2,21.0,pass,0.322631
4974,2017-04-19,P2,No-Seal,NaN,NaN,NaN,AiV2,21.0,pass,0.000000
4975,2017-04-19,P2,Nucleated,NaN,NaN,NaN,AiV2,21.0,pass,NaN
5000,2017-04-20,P2,No-Seal,NaN,NaN,NaN,AiV2,21.0,pass,0.719078
5001,2017-04-20,P2,Outside-Out,NaN,NaN,NaN,AiV2,21.0,pass,0.000000
5002,2017-04-20,P2,Entire-Cell,NaN,NaN,NaN,AiV2,21.0,pass,0.719078
5003,2017-04-20,P2,No-Seal,NaN,NaN,NaN,AiV2,21.0,pass,0.601033


In [171]:
print "Standard internal pass rate:" , float(standard_internal_pass_count)/standard_internal_count

Standard internal pass rate: 0.872031662269


In [172]:
alexa_internal.dropna(subset = ["SM_QC_PF", "Post patch?"], inplace = True)
alexa_internal_count = alexa_internal["Post patch?"].count()
alexa_internal_count

188

In [173]:
alexa_internal_pass = alexa_internal[alexa_internal["SM_QC_PF"] == "pass"]
alexa_internal_pass_count = alexa_internal_pass["Post patch?"].count()
alexa_internal_pass_count

150

In [174]:
print "Alexa internal pass rate:" , float(alexa_internal_pass_count)/alexa_internal_count

Alexa internal pass rate: 0.797872340426


# Rate of SM_QC pass based on internal version and post patch condition

In [175]:
standard_nucleated_pass = pass_only(standard_internal_nuc)
standard_nucleated_pass_pct = calc_pct(standard_nucleated_pass,standard_internal_nuc)
standard_nucleated_pass_pct

0.9151670951156813

In [176]:
alexa_nucleated_pass = pass_only(alexa_internal_nuc)
alexa_nucleated_pass_pct = calc_pct(alexa_nucleated_pass,alexa_internal_nuc)
alexa_nucleated_pass_pct

0.8409090909090909

In [178]:
standard_internal_no_seal = no_seal(standard_internal)
standard_no_seal_pass = pass_only(standard_internal_no_seal)
standard_no_seal_pass_pct = calc_pct(standard_no_seal_pass,standard_internal_no_seal)
standard_no_seal_pass_pct

0.7975206611570248

In [179]:
alexa_internal_no_seal = no_seal(alexa_internal)
alexa_no_seal_pass = pass_only(alexa_internal_no_seal)
alexa_no_seal_pass_pct = calc_pct(alexa_no_seal_pass,alexa_internal_no_seal)
alexa_no_seal_pass_pct

0.4807692307692308